In [5]:
library('aod')
library(ggplot2)
#suppressMessages(library(ROCR))

In [1]:
projects_la <- read.csv('projects_la_moreData_clean.csv')

In [11]:
normalize <- function(x) {
    zeromean <- x-mean(x)
    normalized <- zeromean/sd(x)
    return(normalized)
}

In [7]:
names(projects_la)

[1] "X_projectid"                           
 [2] "school_zip"                            
 [3] "school_metro"                          
 [4] "school_charter"                        
 [5] "school_magnet"                         
 [6] "school_year_round"                     
 [7] "school_kipp"                           
 [8] "school_charter_ready_promise"          
 [9] "teacher_teach_for_america"             
[10] "grade_level"                           
[11] "total_price_excluding_optional_support"
[12] "total_price_including_optional_support"
[13] "students_reached"                      
[14] "eligible_double_your_impact_match"     
[15] "eligible_almost_home_match"            
[16] "funding_status"                        
[17] "funding_status_binary"                 
[18] "resource_type_grouped"                 
[19] "primary_focus_subject_grouped"         
[20] "secondary_focus_subject_grouped"       
[21] "poverty_level_grouped"                 
[22] "teacher_gender"                        
[23] "month_posted"                          
[24] "semester_posted"                       
[25] "previousProposal_Teacher"              
[26] "previousProposal_School"               
[27] "donor_per_zip"                         
[28] "donation_per_zip"                      
[29] "via_giving_page"

In [9]:
categorical_col_names <- c('primary_focus_subject_grouped', 'resource_type_grouped'
                           ,'eligible_double_your_impact_match', 'funding_status',
                           'eligible_almost_home_match', 'semester_posted','via_giving_page')
continuous_col_names <- c('total_price_including_optional_support', 'students_reached',
                          'previousProposal_Teacher','previousProposal_School')

In [12]:
categorical_col <- projects_la[,categorical_col_names]
categorical_col <- sapply(categorical_col,function(col) as.factor(col))
continuous_col <- projects_la[continuous_col_names]
continuous_col <- sapply( continuous_col, function(col) normalize(col) )

In [14]:
features <- cbind.data.frame(categorical_col,continuous_col)

In [15]:
dim(features)
head(features)

[1] 46364    11

,primary_focus_subject_grouped,resource_type_grouped,eligible_double_your_impact_match,funding_status,eligible_almost_home_match,semester_posted,via_giving_page,total_price_including_optional_support,students_reached,previousProposal_Teacher,previousProposal_School
1,other,Books_Supplies,0,completed,0,1st,False,0.8343819,0.030588,0.258303,2.101787
2,health,Technology_other,0,completed,0,1st,False,0.4913609,0.09748147,1.385539,2.101787
3,literacy_math,Books_Supplies,0,completed,0,1st,False,-0.01844136,0.09748147,-0.4814453,-0.6997308
4,health,Technology_other,0,completed,0,1st,False,2.972579,0.09748147,1.385539,2.101787
5,literacy_math,Books_Supplies,0,completed,0,1st,False,-0.1702974,-0.3707729,-0.5518975,-0.3282033
6,literacy_math,Books_Supplies,0,completed,0,1st,False,-0.3153849,-0.103199,-0.4814453,-0.6997308


In [16]:
set.seed(1230)
train <- features[1:33000,]
test <- features[33000:46364,]

In [19]:
mylogit <- glm(as.factor(funding_status) ~ .,data=features,family=binomial())

In [20]:
summary(mylogit)


Call:
glm(formula = as.factor(funding_status) ~ ., family = binomial(), 
    data = features)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-6.3804  -0.5147  -0.3675  -0.2723   3.0632  

Coefficients:
                                             Estimate Std. Error z value
(Intercept)                                 -1.456584   0.050822 -28.660
primary_focus_subject_groupedhealth          0.252313   0.058006   4.350
primary_focus_subject_groupedhistory_civics  0.232788   0.074136   3.140
primary_focus_subject_groupedliteracy_math   0.296831   0.047581   6.238
primary_focus_subject_groupedmusic_art      -0.055717   0.063982  -0.871
primary_focus_subject_groupedother           0.250664   0.086311   2.904
resource_type_groupedTechnology_other        0.445666   0.029301  15.210
resource_type_groupedTrips_Visitor          -1.313081   0.180115  -7.290
eligible_double_your_impact_match1           0.008696   0.030608   0.284
eligible_almost_home_match1                  0

In [21]:
## CIs using profiled log-likelihood
confint(mylogit)

Waiting for profiling to be done...


,2.5 %,97.5 %
(Intercept),-1.556772,-1.35753
primary_focus_subject_groupedhealth,0.1388047,0.366215
primary_focus_subject_groupedhistory_civics,0.08692671,0.3775954
primary_focus_subject_groupedliteracy_math,0.2041392,0.3906783
primary_focus_subject_groupedmusic_art,-0.1812587,0.06958889
primary_focus_subject_groupedother,0.08042153,0.4188511
resource_type_groupedTechnology_other,0.3882789,0.5031411
resource_type_groupedTrips_Visitor,-1.679794,-0.9723825
eligible_double_your_impact_match1,-0.05132334,0.06866366
eligible_almost_home_match1,-0.009319052,0.2116095


In [22]:
## CIs using standard errors
confint.default(mylogit)

,2.5 %,97.5 %
(Intercept),-1.556194,-1.356974
primary_focus_subject_groupedhealth,0.1386228,0.3660037
primary_focus_subject_groupedhistory_civics,0.0874838,0.3780918
primary_focus_subject_groupedliteracy_math,0.2035735,0.3900878
primary_focus_subject_groupedmusic_art,-0.1811199,0.069686
primary_focus_subject_groupedother,0.08149642,0.4198309
resource_type_groupedTechnology_other,0.3882375,0.503095
resource_type_groupedTrips_Visitor,-1.6661,-0.9600614
eligible_double_your_impact_match1,-0.05129446,0.06868678
eligible_almost_home_match1,-0.00836961,0.2125117


In [23]:
## odds ratios and 95% CI
exp(cbind(OR = coef(mylogit), confint(mylogit)))

Waiting for profiling to be done...


,OR,2.5 %,97.5 %
(Intercept),0.233031,0.2108155,0.2572955
primary_focus_subject_groupedhealth,1.286999,1.1489,1.442265
primary_focus_subject_groupedhistory_civics,1.262114,1.090817,1.458773
primary_focus_subject_groupedliteracy_math,1.345587,1.226469,1.477983
primary_focus_subject_groupedmusic_art,0.9458068,0.8342195,1.072067
primary_focus_subject_groupedother,1.284878,1.083744,1.520214
resource_type_groupedTechnology_other,1.56153,1.474441,1.653908
resource_type_groupedTrips_Visitor,0.2689901,0.1864124,0.3781809
eligible_double_your_impact_match1,1.008734,0.9499715,1.071076
eligible_almost_home_match1,1.107462,0.9907242,1.235665


In [ ]:
Y_true <- test['funding_status']
test$funding_status <- NULL

In [109]:
confusionMatrix(Y_true$funding_status,Prediction)

Confusion Matrix and Statistics

           Reference
Prediction  completed expired
  completed      4547     343
  expired        1443     317
                                          
               Accuracy : 0.7314          
                 95% CI : (0.7206, 0.7421)
    No Information Rate : 0.9008          
    P-Value [Acc > NIR] : 1               
                                          
                  Kappa : 0.1375          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.7591          
            Specificity : 0.4803          
         Pos Pred Value : 0.9299          
         Neg Pred Value : 0.1801          
             Prevalence : 0.9008          
         Detection Rate : 0.6838          
   Detection Prevalence : 0.7353          
      Balanced Accuracy : 0.6197          
                                          
       'Positive' Class : completed       
                                       

In [ ]:
sort(varimp(fit))

In [112]:
GLM1 <- glm(as.factor(funding_status) ~ .,data=features,family=binomial())

In [113]:
fit


Call:
glm(formula = as.factor(funding_status) ~ ., family = binomial(), 
    data = features)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-5.4962  -0.7835  -0.6000   0.9091   2.7279  

Coefficients:
                                         Estimate Std. Error z value Pr(>|z|)
(Intercept)                            -1.079e+01  1.124e+02  -0.096  0.92353
teacher_teach_for_america1             -5.775e-01  6.719e-02  -8.595  < 2e-16
resource_typeBooks                      9.696e+00  1.124e+02   0.086  0.93127
resource_typeOther                      9.872e+00  1.124e+02   0.088  0.93002
resource_typeSupplies                   9.780e+00  1.124e+02   0.087  0.93067
resource_typeTechnology                 1.029e+01  1.124e+02   0.092  0.92708
resource_typeTrips                      7.972e+00  1.124e+02   0.071  0.94346
resource_typeVisitors                   9.687e+00  1.124e+02   0.086  0.93133
eligible_double_your_impact_match1     -5.877e-01  3.367e-02 -17.451  < 2e

In [116]:
exp(coef(GLM1))

(Intercept)             teacher_teach_for_america1 
                          4.855213e+04                           1.781636e+00 
                    resource_typeBooks                     resource_typeOther 
                          6.153430e-05                           5.161430e-05 
                 resource_typeSupplies                resource_typeTechnology 
                          5.657178e-05                           3.404230e-05 
                    resource_typeTrips                  resource_typeVisitors 
                          3.448377e-04                           6.208863e-05 
    eligible_double_your_impact_match1            eligible_almost_home_match1 
                          1.799781e+00                           1.871004e+00 
                         month_posted2                          month_posted3 
                          3.449981e-01                           8.943433e-01 
                         month_posted4                          month_posted5 
                          1.055142e+00                           8.953459e-01 
                         month_posted6                          month_posted7 
                          9.011353e-01                           1.067210e+00 
                         month_posted8                          month_posted9 
                          1.921800e+00                           1.272031e+00 
                        month_posted10                         month_posted11 
                          1.326868e+00                           1.377387e+00 
                        month_posted12 total_price_excluding_optional_support 
                          1.552023e+00                           9.996227e-01 
                      students_reached               previousProposal_Teacher 
                          9.998770e-01                           1.001744e+00 
               previousProposal_School 
                          1.000745e+00

In [114]:
confint(GLM1)

Waiting for profiling to be done...
Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities num

,2.5 %,97.5 %
(Intercept),NA,3.877933
teacher_teach_for_america1,-0.7106531,-0.4471813
resource_typeBooks,-4.9727,NA
resource_typeOther,-4.796489,NA
resource_typeSupplies,-4.888799,NA
resource_typeTechnology,-4.381002,NA
resource_typeTrips,-6.683508,NA
resource_typeVisitors,-4.924067,NA
eligible_double_your_impact_match1,-0.6537669,-0.5217592
eligible_almost_home_match1,-0.753924,-0.5015228


In [118]:
?glm

glm {stats},R Documentation
formula,"an object of class ""formula"" (or one that can be coerced to that class): a symbolic description of the model to be fitted. The details of model specification are given under ‘Details’."
family,"a description of the error distribution and link function to be used in the model. For glm this can be a character string naming a family function, a family function or the result of a call to a family function. For glm.fit only the third option is supported. (See family for details of family functions.)"
data,"an optional data frame, list or environment (or object coercible by as.data.frame to a data frame) containing the variables in the model. If not found in data, the variables are taken from environment(formula), typically the environment from which glm is called."
weights,an optional vector of ‘prior weights’ to be used in the fitting process. Should be NULL or a numeric vector.
subset,an optional vector specifying a subset of observations to be used in the fitting process.
na.action,"a function which indicates what should happen when the data contain NAs. The default is set by the na.action setting of options, and is na.fail if that is unset. The ‘factory-fresh’ default is na.omit. Another possible value is NULL, no action. Value na.exclude can be useful."
start,starting values for the parameters in the linear predictor.
etastart,starting values for the linear predictor.
mustart,starting values for the vector of means.
offset,"this can be used to specify an a priori known component to be included in the linear predictor during fitting. This should be NULL or a numeric vector of length equal to the number of cases. One or more offset terms can be included in the formula instead or as well, and if more than one is specified their sum is used. See model.offset."
